In [15]:
import numpy as np
import os
from xml.etree import ElementTree
from xml.etree.ElementTree import XMLParser
from numpy import array
float32 = 'float32'
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook

In [160]:
def drop_dollar(some_str):
    try:
        if some_str[0] == '$':
            return some_str[1::]
        else:
            return some_str
    except:
        pass
    
def rect_diagonal(some_array):
    return [[int(some_array[0][0]), int(some_array[0][1])], [int(some_array[2][0]), int(some_array[2][1])]]    
    
    
def ground_true_answer(xml_file):
    tree = ElementTree.parse(xml_file)
    root = tree.getroot()
    symbols = ['♠','♥','♦️','♣','♦','♥','♠','♣','♠','♠','♣','♣','♦','♦'] 
    suite_list = []
    rang_list = []
    money_list = {}  
    card_list = {}
    for obj in root.findall('object'):
        for i in obj:
            if i.text == 'cards':
                suite =  (obj.findall('attributes')[0].text).split('=')[1]        
                a = (obj.findall('polygon')[0])            
                coord = []
                for i in a.findall('pt'):       
                    pair = []
                    for el in i:                
                        pair+=[int(float(el.text))]
                    coord +=[pair]
                if suite in symbols:
                    suite_list += [[suite, coord]]
                elif suite not in symbols:
                    rang_list += [[suite, coord]]

            elif i.text == 'money':
                money =  (obj.findall('attributes')[0].text).split('=')[1]
                a = (obj.findall('polygon')[0])            
                coord = []
                for i in a.findall('pt'):       
                    pair = []
                    for el in i:                
                        pair+=[int(float(el.text))]
                    coord +=[pair]            
                money_list[drop_dollar(money)]=rect_diagonal(coord)          
            # elif money 
                
      
    for i in suite_list:
        for j in rang_list:
            if int(i[1][0][0])-15 < int(j[1][0][0]) < int(i[1][0][0]) + 15\
                        and int(i[1][0][1])-100 < int(j[1][0][1]) < int(i[1][0][1]) + 100:
                card_list[j[0]+i[0]] = [j[1][0], i[1][2]]
            
    return card_list, money_list


def predict_answer(predict):    
    card_list = {}
    money_list = {}
    symbols = ['♠','♥','♦️','♣','♦','♥','♠','♣'] 
    for i in predict:
        if i[0].startswith('$'):
            money_list[drop_dollar(i[0])] = rect_diagonal(i[1])
        elif len(i[0])==2 and i[0][1] in symbols:
            card_list[i[0]] = rect_diagonal(i[1])
        elif len(i[0])==3 and i[0][2] in symbols:
            card_list[i[0]] = rect_diagonal(i[1])
    return card_list, money_list


def debug(predict_array, ground_dict, predict_dict, crop =False, save_dir=None, image_file=None): 
    wrong_answer = {}
    for key, val in ground_dict.items():
        if key not in predict_dict:     
            wrong_answer[key] = val            
    #print (wrong_answer)
    recall_count = 0
    for key, val in predict_dict.items():
        if key not in ground_dict:     
            recall_count += 1
       
    if crop == False:
        pass
    if crop == True:
        for key, val in wrong_answer.items():
            compare = val
            x = compare[0][0]
            y = compare[0][1]
            for i in predict_array:
                first_point = (rect_diagonal(i[1])[0])
                x_pred = first_point[0]
                y_pred = first_point[1]  
                if x-10<x_pred<x+10 and y-10<y_pred<y+10:
                        area = (val[0][0], val[0][1], val[1][0], val[1][1])
                        wrong = drop_dollar(i[0])
                        name = str(wrong)+'_pred_'+str(key)+'_true_'+'.jpg'
                        image = Image.open(image_file)
                        image.crop(area).save(save_dir+name)
                        #print (wrong_answer, key, val)
    
    precision = round((len(ground_dict)-len(wrong_answer))/len(ground_dict),2)
    
    try:
        recall = round((recall_count/len(predict_dict)),2)
    except ZeroDivisionError:
        recall = 0
    return precision, recall
    #return (round(100 - (round(len(wrong_answer)*100/len(ground_dict),2)),2))

In [130]:
def answer_accuracy(ground_true_xml, predict_list,crop, image_file=None, save_dir=None):    
    ground_true_data = ground_true_answer(ground_true_xml)
    predict_data = predict_answer(predict_list)    
    if crop == False:
        card_accuracy = debug(predict_list, ground_true_data[0],predict_data[0])    
        money_accuracy = debug(predict_list, ground_true_data[1], predict_data[1])  
    elif crop == True:
        card_accuracy = debug(predict_list, ground_true_data[0],predict_data[0],crop, save_dir, image_file)
        money_accuracy = debug(predict_list, ground_true_data[1], predict_data[1], crop,save_dir, image_file)
    
    return card_accuracy, money_accuracy

In [ ]:
npy_data = '/home/alex/Загрузки/predict(2).npy'
new_predict = np.load(npy_data, allow_pickle=True)
new_gt_xml = sorted([os.path.join(new_gt_dir,i) for i in os.listdir(new_gt_dir) if i.endswith('.xml')])
new_gt_jpg = sorted([os.path.join(new_gt_dir,i) for i in os.listdir(new_gt_dir) if i.endswith('.jpg')])
new_gt_dir = '/media/alex/Integral-ASIC/GGPokerOk_GT_3_cat-20201113T152041Z-001/GGPokerOk_GT_3_cat'

In [161]:
for i in range(len(new_gt_xml)):
    print (i, answer_accuracy(new_gt_xml[i], new_predict[i], crop = False))

0 ((0.25, 0.0), (1.0, 0.53))
1 ((0.0, 0), (1.0, 0.46))
2 ((0.11, 0.0), (0.71, 0.67))
3 ((0.0, 0), (0.89, 0.43))
4 ((0.25, 0.0), (0.83, 0.58))
5 ((0.25, 0.0), (0.83, 0.58))
6 ((0.2, 0.0), (1.0, 0.5))
7 ((0.25, 0.33), (0.83, 0.62))
8 ((0.14, 0.0), (1.0, 0.5))
9 ((0.2, 0.0), (1.0, 0.57))
10 ((0.14, 0.0), (1.0, 0.61))
11 ((0.29, 0.0), (0.86, 0.65))
12 ((0.43, 0.0), (0.86, 0.62))
13 ((0.0, 0), (0.86, 0.57))
14 ((0.0, 0), (0.86, 0.6))
15 ((0.0, 0), (1.0, 0.5))
16 ((0.0, 0), (1.0, 0.5))
17 ((0.2, 0.0), (0.83, 0.62))
18 ((0.43, 0.0), (0.67, 0.69))
19 ((0.25, 0.0), (0.83, 0.55))
20 ((0.12, 0.0), (0.57, 0.73))
21 ((0.2, 0.0), (0.89, 0.53))
22 ((0.11, 0.0), (1.0, 0.59))
23 ((0.18, 0.0), (0.8, 0.76))
24 ((0.29, 0.33), (0.86, 0.57))
25 ((0.75, 0.0), (1.0, 0.5))
26 ((0.0, 0), (1.0, 0.71))
27 ((0.33, 0.0), (1.0, 0.5))
28 ((0.0, 0), (1.0, 0.47))
29 ((0.0, 0), (0.86, 0.5))
30 ((0.4, 0.0), (1.0, 0.65))
31 ((0.33, 0.0), (0.88, 0.42))
32 ((0.0, 0), (1.0, 0.33))
33 ((0.0, 0), (1.0, 0.31))
34 ((0.14, 0.0), 